## Concept Map Extraction - Chain-of-thought Prompting

In [1]:
import os
from openai import OpenAI
from io import StringIO
import pandas as pd
from tqdm import tqdm
from src.baselines.cot_baseline import run_gpt, DecomposedLLMBaseline
from src.settings import API_KEY_GPT

In [2]:
MODEL = "gpt-3.5-turbo-0125"
DLLM = DecomposedLLMBaseline(model=MODEL)

In [3]:
FOLDER = "../data/Corpora_Falke/Wiki/test/102"
TEXTS = [x for x in os.listdir(FOLDER) if x.endswith(".txt")]
TEXTS = [open(os.path.join(FOLDER, x)).read() for x in TEXTS]

# Pre-processing
TEXTS = [DLLM.nlp(text) for text in TEXTS]
TEXTS = [[sent.text.strip() for sent in text.sents if sent.text.strip()] for text in TEXTS]
TEXTS = [[DLLM.preprocess(x) for x in sentences] for sentences in TEXTS] 
TEXTS = ["\n".join(x) for x in TEXTS]

TEXTS

['A fascinating, recent discovery appears to have solved one of Jerusalem’s biggest historical mysteries: the location of the Acra, the fortified compound in Jerusalem built by Antiochus Epiphanes, ruler of the Hellenistic Seleucid Empire, following his sack of the city in 168 BCE.\nThe renowned fortress was used to control the Jewish city and to monitor the activities in the temple.\nThe Akra was eventually conquered by the Hasmoneans.',
 '(CNN)Archaeologists believe they have found the remains of the ancient Greek fort of Acra, solving "one of Jerusalem\'s greatest archaeological mysteries.\n" The stronghold ruins were unearthed from beneath a parking lot in Jerusalem, Israel\'s Antiquities Authority said.\nAcra dates back more than 2,000 years, to the time of Greek ruler Antiochus IV Epiphanes.\nExcavation directors Doron Ben-Ami, Yana Tchekhanovets and Salome Cohen called it a "sensational discovery.\n" "The new archaeological finds indicate the establishment of a well-fortified st

## Summary Extraction

In [4]:
print(DLLM.prompt_summary)


        You need to write a summary of the text that will be sent in the following message. The summary should be in the same style as the original text. For instance, focus on the content of the text, and do not start with "this text is about" or equivalent.

        The summary is:
        


In [5]:
SUMMARIES = []
for text in tqdm(TEXTS):
    SUMMARIES.append(run_gpt(
        prompt=DLLM.prompt_summary, content=text))

100%|██████████| 16/16 [00:50<00:00,  3.15s/it]


In [6]:
SUMMARIES

["A recent discovery has potentially solved the mystery of the Acra's location in Jerusalem, a fortified compound built by Antiochus Epiphanes in 168 BCE. The fortress was crucial for controlling the city and monitoring the temple, eventually falling to the Hasmoneans.",
 'Archaeologists have discovered the ancient Greek fort of Acra in Jerusalem, solving a long-standing mystery. The stronghold, dating back over 2,000 years, was found beneath a parking lot and was built to control the city and the Temple Mount. Acra played a significant role during the Maccabean revolt and was eventually recaptured by the Jews. The discovery, including artifacts like sling shots and imported wine jars, sheds light on the history of the fort and its inhabitants.',
 "Researchers from the Israel Antiquities Authority have discovered the remains of the Acra stronghold in Jerusalem, built by the Greeks over 2,000 years ago to control the Temple. Evidence of the Hasmonean attempts to conquer the stronghold h

## Entity Extraction

In [7]:
print(DLLM.prompt_entity)


        You need to extract all the entities from the text that will be sent in the following message. You must not introduce a hierarchy over the entities, such as Person or Concept. Each entity should appear in the text as is.

        In your answer, you must give the output in a .csv file with the columns `entity` and `surface`. `entity` contains one label, whereas `surface` contains all the surface forms of that entity. The columns are separated by `;`, and the surface forms in the `surface` column by `,`.

        The output is:
        ```csv
        ```
        


In [8]:
ENTITIES = []
for text in tqdm(SUMMARIES):
    ENTITIES.append(run_gpt(
        prompt=DLLM.prompt_entity, content=text))

100%|██████████| 16/16 [00:46<00:00,  2.92s/it]


In [9]:
ENTITIES

['```csv\nentity;surface\nAcra;Acra\nJerusalem;Jerusalem\nAntiochus Epiphanes;Antiochus Epiphanes\n168 BCE;168 BCE\nHasmoneans;Hasmoneans\n```',
 '```csv\nentity;surface\nArchaeologists;Archaeologists\nancient Greek fort;ancient Greek fort, fort of Acra\nAcra;Acra\nJerusalem;Jerusalem\nstronghold;stronghold\nTemple Mount;Temple Mount\nMaccabean revolt;Maccabean revolt\nJews;Jews\ndiscovery;discovery\nartifacts;artifacts\nsling shots;sling shots\nimported wine jars;imported wine jars\nhistory;history\ninhabitants;inhabitants\n```',
 '```csv\nentity;surface\nIsrael Antiquities Authority;Researchers from the Israel Antiquities Authority\nAcra stronghold;Acra stronghold\nJerusalem;Jerusalem\nGreeks;Greeks\nTemple;Temple\nHasmonean;Hasmonean\nCity of David National Park;City of David National Park\nMaccabean;Maccabean\nlead sling shots;lead sling shots\nbronze arrowheads;bronze arrowheads\ncitadel;citadel\n```',
 "```csv\nentity;surface\nArchaeologists;Archaeologists\nJerusalem;Jerusalem\nA

In [10]:
print(DLLM.prompt_group_entity)


        You need to group all the entities from the .csv that will be sent in the following messages into one file. 

        In your answer, you must give the output in a .csv file with the columns `entity` and `surface`. `entity` contains one label, whereas `surface` contains all the surface forms of that entity. The columns are separated by `;`, and the surface forms in the `surface` column by `,`.

        The output is:
        ```csv
        ```
        


In [11]:
GROUPED_ENTITIES = run_gpt(
    prompt=DLLM.prompt_group_entity, content=ENTITIES)

In [12]:
print(GROUPED_ENTITIES)

```csv
entity;surface
Acra;Acra, the Acra, Fortress of Acra, Acra fortress, Acra stronghold, Acra fortress system
Jerusalem;Jerusalem
Antiochus Epiphanes;Antiochus Epiphanes, King Antiochus IV, Antiochus IV Epiphanes, Seleucid Greek king Antiochus Epiphanes
168 BCE;168 BCE
Hasmoneans;Hasmoneans, Hasmonean, Hasmonean dynasty, The Hasmoneans, Hasmonean Kingdom
Archaeologists;Archaeologists, Researchers from the Israel Antiquities Authority, Israeli archaeologist
ancient Greek fort;ancient Greek fort, fort of Acra
stronghold;stronghold
Temple Mount;Temple Mount, Solomon’s Temple Mount, Temple Mount, Temple Mount, The Temple Mount area
Maccabean revolt;Maccabean revolt, Maccabean Revolt, Maccabean Revolt
Jews;Jews, Jewish, observant Jews
discovery;discovery
artifacts;artifacts
sling shots;sling shots, lead sling shots, lead sling stones
imported wine jars;imported wine jars
history;history, Jerusalem's history
inhabitants;inhabitants
Greeks;Greeks
Temple;Temple
City of David National Park;

## Relation Extraction

In [13]:
print(DLLM.prompt_relation)


        You need to extract all the relations from the text that will be sent in the following message. Each relation is in the form of a triple (subject, predicate, object), where subject and object are entities that you identified in the previous step. `subject` and `object` should be from the list of entities you will be sent. Each relation must be unique, no repetitions.

        In your answer, you must give the output in a .csv file with the columns with the columns `subject`,  `predicate` and `object`. The columns are separated by `;`.

        The output is:
        ```csv
        ```
        


In [14]:
INFO = {"entities": GROUPED_ENTITIES}
RELATIONS = []
for text in tqdm(SUMMARIES):
    RELATIONS.append(run_gpt(
        prompt=DLLM.prompt_relation, content=text, **INFO))

100%|██████████| 16/16 [01:05<00:00,  4.11s/it]


In [15]:
RELATIONS

['```csv\nsubject;predicate;object\nAcra;located in;Jerusalem\nAcra;built by;Antiochus Epiphanes\nAcra;fallen to;Hasmoneans\nfortress;was crucial for;controlling the city\nfortress;was crucial for;monitoring the temple\nfortress;was crucial for;controlling the city\nfortress;was crucial for;monitoring the temple\nAntiochus Epiphanes;built;Acra\nAntiochus Epiphanes;built;fortress\nAntiochus Epiphanes;built;fortified compound\nfortress;was crucial for;controlling the city\nfortress;was crucial for;monitoring the temple\nfortress;was crucial for;controlling the city\nfortress;was crucial for;monitoring the temple\n```',
 '```csv\nArchaeologists;discovered;ancient Greek fort\nAcra;solved;mystery\nfort;dating back over 2,000 years\nfort;found;beneath parking lot\nfort;built;to control city and Temple Mount\nAcra;played role;Maccabean revolt\nAcra;recaptured;Jews\ndiscovery;sheds light on;history of fort and inhabitants\ndiscovery;included;artifacts like sling shots and imported wine jars\n`

In [16]:
print(DLLM.prompt_group_relation)


        You need to group all the relations from the .csv that will be sent in the following messages. If there are duplicates, only keep one relation and ensure that each relation is unique. 

        In your answer, you must give the output in a .csv file with the columns with the columns `subject`,  `predicate` and `object`. The columns are separated by `;`.

        The output is:
        ```csv
        ```
        


In [17]:
GROUPED_RELATIONS = run_gpt(
    prompt=DLLM.prompt_group_relation, content=RELATIONS)

In [18]:
print(GROUPED_RELATIONS)

```csv
subject;predicate;object
Acra;built by;Antiochus Epiphanes
Acra;located in;Jerusalem
Acra;solved;mystery
fortress;was crucial for;controlling the city
fortress;was crucial for;monitoring the temple
Antiochus Epiphanes;built;Acra
Antiochus Epiphanes;built;fortress
Antiochus Epiphanes;built;fortified compound
Israel Antiquities Authority;discovered;Acra stronghold
Acra stronghold;built by;Greeks
Acra stronghold;located in;Jerusalem
Hasmoneans;attempted to conquer;Acra stronghold
excavation;uncovered;massive wall
excavation;uncovered;tower base
excavation;uncovered;defensive embankment
settlement;existed before;Maccabean uprising
defenses of stronghold;included;lead sling shots
defenses of stronghold;included;bronze arrowheads
discovery;provides insight into;history and non-Jewish identity
citadel's inhabitants;had;non-Jewish identity
Acra;built in;186 BC
Acra;served as military post against;Jewish people
Acra;monitored;Temple Mount activities
Acra;demolished in;141 BC
Temple Mount

## Importance Ranking

In [19]:
print(DLLM.prompt_ir)


        You will be provided with a set of triples, where each triple consists of a subject, predicate, and object. Your task is to remove redundant triples, and to extract a subset of these triples that represent the most important information from the original set.

        In your answer, you must give the output in a .csv file with the columns with the columns `subject`,  `predicate` and `object`. The columns are separated by `;`.

        The output is:
        ```csv
        ```
        


In [21]:
OUTPUT = run_gpt(
    prompt=DLLM.prompt_ir, content=GROUPED_RELATIONS)

In [23]:
print(OUTPUT)

```csv
subject;predicate;object
Acra;built by;Antiochus Epiphanes
Acra;located in;Jerusalem
Acra;solved;mystery fortress
Acra;was crucial for;controlling the city
Acra;was crucial for;monitoring the temple
Antiochus Epiphanes;built;fortress
Antiochus Epiphanes;built;fortified compound
Israel Antiquities Authority;discovered;Acra stronghold
Acra stronghold;built by;Greeks
Acra stronghold;located in;Jerusalem
Hasmoneans;attempted to conquer;Acra stronghold
excavation;uncovered;massive wall
excavation;uncovered;tower base
excavation;uncovered;defensive embankment
settlement;existed before;Maccabean uprising
defenses of stronghold;included;lead sling shots
defenses of stronghold;included;bronze arrowheads
discovery;provides insight into;history and non-Jewish identity
citadel's inhabitants;had;non-Jewish identity
Acra;built in;186 BC
Acra;served as military post against;Jewish people
Acra;monitored;Temple Mount activities
Acra;demolished in;141 BC
Temple Mount;extended to the south to conc

In [29]:
OUTPUT = OUTPUT.replace("```csv", "").replace("```", "")
pd.read_csv(StringIO(OUTPUT), sep=";", on_bad_lines='skip')  

,subject,predicate,object
0,Acra,built by,Antiochus Epiphanes
1,Acra,located in,Jerusalem
2,Acra,solved,mystery fortress
3,Acra,was crucial for,controlling the city
4,Acra,was crucial for,monitoring the temple
...,...,...,...
109,Simon Maccabeus,became,strategus
110,Simon Maccabeus,became,ethnarch of the Jews
111,Simon Maccabeus,marked,the beginning of the Hasmonean dynasty
112,Simon Maccabeus,reign ended tragically with,violent death in 135 B.C.
